In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
from copy import deepcopy
import os

file_to_load = "salaries-by-region.csv"

In [2]:
#read region data
region_data = pd.read_csv(file_to_load)

#Extract data by Region from the Master DF
Southern_Region = region_data.loc[region_data["Region"] == "Southern", :]

#List salary change over the lenght of mid-career by region; from school with hightest Starting_Mid to lowest Mid_90th
Southern_Sort = Southern_Region.sort_values(['Starting Median Salary', 'School Name'], ascending=[False, True])

#Clean DF to the areas of focus
South_Clean = Southern_Sort[["School Name", "Region", "Starting Median Salary", "Mid-Career Median Salary", "Mid-Career 90th Percentile Salary"]]
South_Clean = South_Clean.dropna()

#Index data for Region Column
South_Clean.set_index('Region', inplace=True)

#Add unique index value row
df = South_Clean.reset_index()
df.head()

,Region,School Name,Starting Median Salary,Mid-Career Median Salary,Mid-Career 90th Percentile Salary
0,Southern,Rice University,"$64,000.00","$110,000.00","$216,000.00"
1,Southern,Duke University,"$58,900.00","$106,000.00","$218,000.00"
2,Southern,Georgia Institute of Technology,"$58,300.00","$106,000.00","$183,000.00"
3,Southern,Georgetown University,"$55,000.00","$110,000.00","$248,000.00"
4,Southern,Virginia Polytechnic Institute and State Unive...,"$53,500.00","$95,400.00","$163,000.00"


In [3]:
df.columns = ['REGION','SCHOOL NAME','STARTING AVG','MID CAREER AVG ','TOP 10% AVG']
df.head()

,REGION,SCHOOL NAME,STARTING AVG,MID CAREER AVG,TOP 10% AVG
0,Southern,Rice University,"$64,000.00","$110,000.00","$216,000.00"
1,Southern,Duke University,"$58,900.00","$106,000.00","$218,000.00"
2,Southern,Georgia Institute of Technology,"$58,300.00","$106,000.00","$183,000.00"
3,Southern,Georgetown University,"$55,000.00","$110,000.00","$248,000.00"
4,Southern,Virginia Polytechnic Institute and State Unive...,"$53,500.00","$95,400.00","$163,000.00"


In [4]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [5]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [6]:
base_url = 'https://www.payscale.com/cost-of-living-calculator/'

In [7]:
#Create list of links to scrape 
city_list= ['Georgia-Atlanta', 'Texas-Austin','North-Carolina-Charlotte', 'Texas-Dallas',\
            'Texas-Houston', 'Florida-Miami', 'Tennessee-Nashville', 'Florida-Orlando', 'North-Carolina-Raleigh', 'Florida-Tampa', 'District-of-Columbia-Washington']
count = 0
pages = []
for page in city_list:
    page_url = f"{base_url}{city_list[count]}"
    count = count + 1
    pages.append(page_url)
    print(page_url)
    
    

https://www.payscale.com/cost-of-living-calculator/Georgia-Atlanta
https://www.payscale.com/cost-of-living-calculator/Texas-Austin
https://www.payscale.com/cost-of-living-calculator/North-Carolina-Charlotte
https://www.payscale.com/cost-of-living-calculator/Texas-Dallas
https://www.payscale.com/cost-of-living-calculator/Texas-Houston
https://www.payscale.com/cost-of-living-calculator/Florida-Miami
https://www.payscale.com/cost-of-living-calculator/Tennessee-Nashville
https://www.payscale.com/cost-of-living-calculator/Florida-Orlando
https://www.payscale.com/cost-of-living-calculator/North-Carolina-Raleigh
https://www.payscale.com/cost-of-living-calculator/Florida-Tampa
https://www.payscale.com/cost-of-living-calculator/District-of-Columbia-Washington


In [8]:
headers = ['Median Home Price','Median Rent','Energy Bill','Phone Bill','Gas','Loaf of Bread', \
            'Gallon of Milk','Carton of Eggs','Bunch of Bananas','Hamburger',"Doctor's Visit",\
            'Dentist Visit','Optometrist Visit','Rx Drug','Veterinary Visit']

In [9]:
count = 0
lifestyle_df = pd.DataFrame()
for page in pages:
    browser.visit(page)
    south_region = browser.html
    southern_soup = bs(south_region, 'html.parser')
    values = []
    for info in southern_soup.find_all('div', class_='itemized__item-container'):   
        for v in info.find_all('div', class_='itemized__item-value'):
            value = v.text
            values.append(deepcopy(value))
        
    style_df = pd.DataFrame([{headers[x]:values[x] for x in range(len(headers))}])
    lifestyle_df = lifestyle_df.append(deepcopy(style_df))  
    

In [10]:
df = pd.DataFrame(lifestyle_df)
df['STATES']= city_list


In [11]:
Southern_data = df.set_index('STATES')
Southern_data

,Median Home Price,Median Rent,Energy Bill,Phone Bill,Gas,Loaf of Bread,Gallon of Milk,Carton of Eggs,Bunch of Bananas,Hamburger,Doctor's Visit,Dentist Visit,Optometrist Visit,Rx Drug,Veterinary Visit
STATES,,,,,,,,,,,,,,,
Georgia-Atlanta,"$327,568","$1,026 / month",$143.01 / month,$152.45 / month,$2.64 / gallon,$3.50,$2.04,$1.90,$3.53,$4.46,$119.01,$102.24,$111.52,$474.08,$52.50
Texas-Austin,"$353,246","$1,107 / month",$147.04 / month,$156.74 / month,$2.44 / gallon,$2.92,$1.70,$1.59,$2.95,$3.72,$110.05,$94.54,$103.12,$438.35,$51.88
North-Carolina-Charlotte,"$296,685",$929.39 / month,$153.91 / month,$164.06 / month,$2.53 / gallon,$3.12,$1.82,$1.70,$3.15,$3.98,$114.14,$98.06,$106.96,$454.67,$52.34
Texas-Dallas,"$364,003","$1,140 / month",$174.87 / month,$186.40 / month,$2.59 / gallon,$3.16,$1.84,$1.72,$3.19,$4.02,$112.04,$96.25,$104.98,$446.29,$52.65
Texas-Houston,"$363,656","$1,139 / month",$185.43 / month,$197.66 / month,$2.54 / gallon,$2.89,$1.68,$1.57,$2.92,$3.68,$100.97,$86.74,$94.61,$402.19,$48.64
Florida-Miami,"$498,986","$1,563 / month",$165.31 / month,$176.22 / month,$2.93 / gallon,$3.51,$2.04,$1.91,$3.54,$4.47,$107.39,$92.26,$100.63,$427.77,$52.19
Tennessee-Nashville,"$316,117",$990.26 / month,$150.73 / month,$160.67 / month,$2.62 / gallon,$3.23,$1.88,$1.75,$3.26,$4.11,$90.45,$77.70,$84.76,$360.30,$53.37
Florida-Orlando,"$308,483",$966.34 / month,$186.44 / month,$198.74 / month,$2.45 / gallon,$3.40,$1.98,$1.85,$3.43,$4.33,$97.65,$83.89,$91.50,$388.96,$48.69
North-Carolina-Raleigh,"$241,512",$756.55 / month,$161.62 / month,$172.29 / month,$2.39 / gallon,$3.77,$2.20,$2.05,$3.80,$4.80,$113.92,$97.87,$106.75,$453.79,$54.82


In [12]:
for x in Southern_data:
    Southern_data[x] = Southern_data[x].str.replace("$","")
    Southern_data[x] = Southern_data[x].str.replace("month","")
    Southern_data[x] = Southern_data[x].str.replace("/","")
    Southern_data[x] = Southern_data[x].str.replace("gallon","")
    Southern_data[x] = Southern_data[x].str.replace(",","")
    Southern_data[x] = pd.to_numeric(Southern_data[x])

Southern_data.head()

,Median Home Price,Median Rent,Energy Bill,Phone Bill,Gas,Loaf of Bread,Gallon of Milk,Carton of Eggs,Bunch of Bananas,Hamburger,Doctor's Visit,Dentist Visit,Optometrist Visit,Rx Drug,Veterinary Visit
STATES,,,,,,,,,,,,,,,
Georgia-Atlanta,327568,1026.00,143.01,152.45,2.64,3.50,2.04,1.90,3.53,4.46,119.01,102.24,111.52,474.08,52.50
Texas-Austin,353246,1107.00,147.04,156.74,2.44,2.92,1.70,1.59,2.95,3.72,110.05,94.54,103.12,438.35,51.88
North-Carolina-Charlotte,296685,929.39,153.91,164.06,2.53,3.12,1.82,1.70,3.15,3.98,114.14,98.06,106.96,454.67,52.34
Texas-Dallas,364003,1140.00,174.87,186.40,2.59,3.16,1.84,1.72,3.19,4.02,112.04,96.25,104.98,446.29,52.65
Texas-Houston,363656,1139.00,185.43,197.66,2.54,2.89,1.68,1.57,2.92,3.68,100.97,86.74,94.61,402.19,48.64


In [13]:
#Renamed columns
Southern_data.columns = ['MORTGAGE','RENT','ELECTRICITY','PHONE','GAS','BREAD','MILK','EGGS','BANANAS','POLTRY','PHYSICIAN', 'DENTIST','OPTOMISTRIST', 'RX','VET']
Southern_data['HLTH & WELL']=Southern_data['PHYSICIAN'] + Southern_data['DENTIST'] + Southern_data['OPTOMISTRIST'] + Southern_data['RX']
Southern_data['EAT-IN']=Southern_data['BREAD'] + Southern_data['MILK'] + Southern_data['BANANAS'] + Southern_data['POLTRY']
Southern_data = Southern_data.drop(['BREAD','MILK','EGGS','BANANAS','POLTRY','PHYSICIAN', 'DENTIST', 'OPTOMISTRIST','RX'], axis=1)
Southern_data


,MORTGAGE,RENT,ELECTRICITY,PHONE,GAS,VET,HLTH & WELL,EAT-IN
STATES,,,,,,,,
Georgia-Atlanta,327568,1026.00,143.01,152.45,2.64,52.50,806.85,13.53
Texas-Austin,353246,1107.00,147.04,156.74,2.44,51.88,746.06,11.29
North-Carolina-Charlotte,296685,929.39,153.91,164.06,2.53,52.34,773.83,12.07
Texas-Dallas,364003,1140.00,174.87,186.40,2.59,52.65,759.56,12.21
Texas-Houston,363656,1139.00,185.43,197.66,2.54,48.64,684.51,11.17
Florida-Miami,498986,1563.00,165.31,176.22,2.93,52.19,728.05,13.56
Tennessee-Nashville,316117,990.26,150.73,160.67,2.62,53.37,613.21,12.48
Florida-Orlando,308483,966.34,186.44,198.74,2.45,48.69,662.00,13.14
North-Carolina-Raleigh,241512,756.55,161.62,172.29,2.39,54.82,772.33,14.57


In [14]:
#Added Region column with "Southern" values in preparation to join with other regions.
region = 'Southern'
Southern_data['REGION'] = [region for _ in range(len(Southern_data))]
Southern_data


,MORTGAGE,RENT,ELECTRICITY,PHONE,GAS,VET,HLTH & WELL,EAT-IN,REGION
STATES,,,,,,,,,
Georgia-Atlanta,327568,1026.00,143.01,152.45,2.64,52.50,806.85,13.53,Southern
Texas-Austin,353246,1107.00,147.04,156.74,2.44,51.88,746.06,11.29,Southern
North-Carolina-Charlotte,296685,929.39,153.91,164.06,2.53,52.34,773.83,12.07,Southern
Texas-Dallas,364003,1140.00,174.87,186.40,2.59,52.65,759.56,12.21,Southern
Texas-Houston,363656,1139.00,185.43,197.66,2.54,48.64,684.51,11.17,Southern
Florida-Miami,498986,1563.00,165.31,176.22,2.93,52.19,728.05,13.56,Southern
Tennessee-Nashville,316117,990.26,150.73,160.67,2.62,53.37,613.21,12.48,Southern
Florida-Orlando,308483,966.34,186.44,198.74,2.45,48.69,662.00,13.14,Southern
North-Carolina-Raleigh,241512,756.55,161.62,172.29,2.39,54.82,772.33,14.57,Southern


In [15]:
#Reorderd columns
Southern_data = Southern_data[['MORTGAGE', 'RENT', 'ELECTRICITY','GAS', 'EAT-IN','PHONE','HLTH & WELL', 'VET', 'REGION']]
Southern_data

,MORTGAGE,RENT,ELECTRICITY,GAS,EAT-IN,PHONE,HLTH & WELL,VET,REGION
STATES,,,,,,,,,
Georgia-Atlanta,327568,1026.00,143.01,2.64,13.53,152.45,806.85,52.50,Southern
Texas-Austin,353246,1107.00,147.04,2.44,11.29,156.74,746.06,51.88,Southern
North-Carolina-Charlotte,296685,929.39,153.91,2.53,12.07,164.06,773.83,52.34,Southern
Texas-Dallas,364003,1140.00,174.87,2.59,12.21,186.40,759.56,52.65,Southern
Texas-Houston,363656,1139.00,185.43,2.54,11.17,197.66,684.51,48.64,Southern
Florida-Miami,498986,1563.00,165.31,2.93,13.56,176.22,728.05,52.19,Southern
Tennessee-Nashville,316117,990.26,150.73,2.62,12.48,160.67,613.21,53.37,Southern
Florida-Orlando,308483,966.34,186.44,2.45,13.14,198.74,662.00,48.69,Southern
North-Carolina-Raleigh,241512,756.55,161.62,2.39,14.57,172.29,772.33,54.82,Southern


In [16]:
Southern_data.to_csv("southern.csv")

In [17]:
Southern_data.mean(axis = 0)

MORTGAGE       381542.272727
RENT             1195.221818
ELECTRICITY       165.952727
GAS                 2.602727
EAT-IN             12.821818
PHONE             176.900000
HLTH & WELL       728.045455
VET                51.586364
dtype: float64

In [18]:
#Transpose the DataFrame
southern_avg = pd.DataFrame(Southern_data.mean())
southern_avg_t = southern_avg.T
southern_avg_t 

,MORTGAGE,RENT,ELECTRICITY,GAS,EAT-IN,PHONE,HLTH & WELL,VET
0,381542.272727,1195.221818,165.952727,2.602727,12.821818,176.9,728.045455,51.586364


In [19]:

mid_west_df = pd.read_csv("mid_west.csv")
southern_df_csv = pd.read_csv("southern.csv")

In [20]:
mid_west_df.head()

,STATES,MORTGAGE,RENT,ELECTRICITY,GAS,EAT-IN,PHONE,HLTH & WELL,VET,REGION
0,Illinois-Chicago,539932,1691.00,151.23,3.30,15.89,161.21,760.32,59.09,Mid-West
1,Ohio-Cincinnati,264414,828.29,159.95,2.63,13.40,170.50,738.55,52.45,Mid-West
2,Ohio-Cleveland,306401,959.82,175.54,2.68,16.32,187.12,759.56,54.56,Mid-West
3,Michigan-Detroit,326527,1023.00,171.18,2.82,13.32,182.47,707.02,50.03,Mid-West
4,Missouri-Kansas-City,319587,1001.00,180.74,2.57,14.71,192.66,742.30,53.48,Mid-West


In [21]:
southern_df_csv.head()

,STATES,MORTGAGE,RENT,ELECTRICITY,GAS,EAT-IN,PHONE,HLTH & WELL,VET,REGION
0,Georgia-Atlanta,327568,1026.00,143.01,2.64,13.53,152.45,806.85,52.50,Southern
1,Texas-Austin,353246,1107.00,147.04,2.44,11.29,156.74,746.06,51.88,Southern
2,North-Carolina-Charlotte,296685,929.39,153.91,2.53,12.07,164.06,773.83,52.34,Southern
3,Texas-Dallas,364003,1140.00,174.87,2.59,12.21,186.40,759.56,52.65,Southern
4,Texas-Houston,363656,1139.00,185.43,2.54,11.17,197.66,684.51,48.64,Southern


In [22]:
Mid_South = mid_west_df.append(southern_df_csv,ignore_index = True)
Mid_South

,STATES,MORTGAGE,RENT,ELECTRICITY,GAS,EAT-IN,PHONE,HLTH & WELL,VET,REGION
0,Illinois-Chicago,539932,1691.00,151.23,3.30,15.89,161.21,760.32,59.09,Mid-West
1,Ohio-Cincinnati,264414,828.29,159.95,2.63,13.40,170.50,738.55,52.45,Mid-West
2,Ohio-Cleveland,306401,959.82,175.54,2.68,16.32,187.12,759.56,54.56,Mid-West
3,Michigan-Detroit,326527,1023.00,171.18,2.82,13.32,182.47,707.02,50.03,Mid-West
4,Missouri-Kansas-City,319587,1001.00,180.74,2.57,14.71,192.66,742.30,53.48,Mid-West
5,Wisconsin-Milwaukee,325833,1021.00,179.73,2.49,14.37,191.59,851.13,48.74,Mid-West
6,Minnesota-Minneapolis,363656,1139.00,152.74,2.82,15.65,162.81,784.34,55.07,Mid-West
7,Missouri-St.-Louis,252269,790.25,192.31,2.62,15.34,204.99,766.32,49.98,Mid-West
8,Georgia-Atlanta,327568,1026.00,143.01,2.64,13.53,152.45,806.85,52.50,Southern
9,Texas-Austin,353246,1107.00,147.04,2.44,11.29,156.74,746.06,51.88,Southern


In [23]:
cali_df = pd.read_csv("calif.csv")
western_df= pd.read_csv("westerns.csv")

In [24]:
western_df

,STATES,MORTGAGE,RENT,ELECTRICITY,GAS,EAT-IN,PHONE,HLTH & WELL,VET,REGION
0,Washington-Seattle,673874,2111,206.39,3.47,16.31,220.00,932.20,69.64,Western
1,Oregon-Portland,620783,1945,130.77,2.78,14.82,139.40,803.85,60.53,Western
2,Arizona-Phoenix,330344,1035,161.12,2.39,12.53,171.75,733.29,48.23,Western
3,Colorado-Denver,463245,1451,156.76,2.75,12.60,167.10,790.34,55.28,Western


In [25]:
cali_df


,STATES,MORTGAGE,RENT,ELECTRICITY,GAS,EAT-IN,PHONE,HLTH & WELL,VET,REGION
0,California-San-Francisco,4.32,2.33,114.23,182.92,1146845.28,132.96,4442.21,62.84,California
1,California-Los-Angeles,3.77,2.03,104.53,180.74,788393.32,121.67,3261.33,54.97,California
2,California-San-Diego,3.79,2.04,102.24,192.64,817194.31,119.01,3350.95,52.55,California
3,California-San-Jose,4.06,2.18,110.42,210.75,860916.27,128.53,3554.09,55.07,California


In [26]:
Western_Cali = western_df.append(cali_df,ignore_index = True)
Western_Cali

,STATES,MORTGAGE,RENT,ELECTRICITY,GAS,EAT-IN,PHONE,HLTH & WELL,VET,REGION
0,Washington-Seattle,673874.00,2111.00,206.39,3.47,16.31,220.00,932.20,69.64,Western
1,Oregon-Portland,620783.00,1945.00,130.77,2.78,14.82,139.40,803.85,60.53,Western
2,Arizona-Phoenix,330344.00,1035.00,161.12,2.39,12.53,171.75,733.29,48.23,Western
3,Colorado-Denver,463245.00,1451.00,156.76,2.75,12.60,167.10,790.34,55.28,Western
4,California-San-Francisco,4.32,2.33,114.23,182.92,1146845.28,132.96,4442.21,62.84,California
5,California-Los-Angeles,3.77,2.03,104.53,180.74,788393.32,121.67,3261.33,54.97,California
6,California-San-Diego,3.79,2.04,102.24,192.64,817194.31,119.01,3350.95,52.55,California
7,California-San-Jose,4.06,2.18,110.42,210.75,860916.27,128.53,3554.09,55.07,California


In [27]:
Sth_West_Mid_Cali = Mid_South.append(Western_Cali,ignore_index = True)
Sth_West_Mid_Cali


,STATES,MORTGAGE,RENT,ELECTRICITY,GAS,EAT-IN,PHONE,HLTH & WELL,VET,REGION
0,Illinois-Chicago,539932.00,1691.00,151.23,3.30,15.89,161.21,760.32,59.09,Mid-West
1,Ohio-Cincinnati,264414.00,828.29,159.95,2.63,13.40,170.50,738.55,52.45,Mid-West
2,Ohio-Cleveland,306401.00,959.82,175.54,2.68,16.32,187.12,759.56,54.56,Mid-West
3,Michigan-Detroit,326527.00,1023.00,171.18,2.82,13.32,182.47,707.02,50.03,Mid-West
4,Missouri-Kansas-City,319587.00,1001.00,180.74,2.57,14.71,192.66,742.30,53.48,Mid-West
5,Wisconsin-Milwaukee,325833.00,1021.00,179.73,2.49,14.37,191.59,851.13,48.74,Mid-West
6,Minnesota-Minneapolis,363656.00,1139.00,152.74,2.82,15.65,162.81,784.34,55.07,Mid-West
7,Missouri-St.-Louis,252269.00,790.25,192.31,2.62,15.34,204.99,766.32,49.98,Mid-West
8,Georgia-Atlanta,327568.00,1026.00,143.01,2.64,13.53,152.45,806.85,52.50,Southern
9,Texas-Austin,353246.00,1107.00,147.04,2.44,11.29,156.74,746.06,51.88,Southern


In [28]:
NE = pd.read_csv('Northeastern.csv')
NE

,States,MORTGAGE,RENT,ELECTRICITY,GAS,EAT-IN,PHONE,HLTH & WELL,VET,REGION
0,Massachusetts-Boston,717943,2249.00,236.57,2.89,15.52,252.17,1014.76,66.81,Northeast
1,New-York-New-York,1628124,5100.00,210.08,3.47,18.81,223.94,863.15,76.07,Northeast
2,Pennsylvania-Pittsburgh,317505,994.61,184.43,2.78,16.21,196.59,704.03,50.13,Northeast
3,Pennsylvania-Philadelphia,449365,1408.00,208.74,3.00,17.05,222.51,793.34,55.18,Northeast
4,Maryland-Baltimore,509743,1597.00,179.56,2.69,15.74,191.41,687.52,54.40,Northeast


In [29]:
NE_Sth_West_Mid_Cali = Sth_West_Mid_Cali.append(NE,ignore_index = True)
NE_Sth_West_Mid_Cali

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:7116: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,EAT-IN,ELECTRICITY,GAS,HLTH & WELL,MORTGAGE,PHONE,REGION,RENT,STATES,States,VET
0,15.89,151.23,3.30,760.32,539932.00,161.21,Mid-West,1691.00,Illinois-Chicago,NaN,59.09
1,13.40,159.95,2.63,738.55,264414.00,170.50,Mid-West,828.29,Ohio-Cincinnati,NaN,52.45
2,16.32,175.54,2.68,759.56,306401.00,187.12,Mid-West,959.82,Ohio-Cleveland,NaN,54.56
3,13.32,171.18,2.82,707.02,326527.00,182.47,Mid-West,1023.00,Michigan-Detroit,NaN,50.03
4,14.71,180.74,2.57,742.30,319587.00,192.66,Mid-West,1001.00,Missouri-Kansas-City,NaN,53.48
5,14.37,179.73,2.49,851.13,325833.00,191.59,Mid-West,1021.00,Wisconsin-Milwaukee,NaN,48.74
6,15.65,152.74,2.82,784.34,363656.00,162.81,Mid-West,1139.00,Minnesota-Minneapolis,NaN,55.07
7,15.34,192.31,2.62,766.32,252269.00,204.99,Mid-West,790.25,Missouri-St.-Louis,NaN,49.98
8,13.53,143.01,2.64,806.85,327568.00,152.45,Southern,1026.00,Georgia-Atlanta,NaN,52.50
9,11.29,147.04,2.44,746.06,353246.00,156.74,Southern,1107.00,Texas-Austin,NaN,51.88
